### Import

In [24]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
import matplotlib.pyplot as plt

import copy
from typing import Sequence, Callable
from pennylane.tape import QuantumTape
# from pennylane import transform

A_to_au_conversion = 1.8897259885789

active_electrons = 2
active_orbitals = 2

### Structure

In [2]:
symbols = ["N", "H", "H", "N", "H", "H"]
# ground state coordinate: 
geometry = np.array([0.0, 0.0, 0.0 , 0.0, 0.0, 1.015264, 0.978541, 0.0, -0.270591, -0.627449, 1.276052, -0.477492 , -0.897827, 1.825923, 0.332013 , 0.080714, 1.825923, -0.953842])*A_to_au_conversion

### Project Setup

##### Generate qubit hamiltonian of the molecule

In [3]:
H, qubits = qchem.molecular_hamiltonian(
    symbols,
    geometry,
    active_electrons=active_electrons,
    active_orbitals=active_orbitals,
    # method="pyscf"
)

In [4]:
H

<Hamiltonian: terms=27, wires=[0, 1, 2, 3]>


In [5]:
print(H)

  (-109.09386199364073) [I0]
+ (-0.18501660577737972) [Z3]
+ (-0.18501660577737963) [Z2]
+ (0.036240119907183815) [Z1]
+ (0.03624011990718384) [Z0]
+ (0.0003125962232503304) [Y0 Y2]
+ (0.0003125962232503304) [X0 X2]
+ (0.00380177503471649) [Y1 Y3]
+ (0.00380177503471649) [X1 X3]
+ (0.11823111975967418) [Z0 Z2]
+ (0.11823111975967418) [Z1 Z3]
+ (0.12599593089508843) [Z0 Z3]
+ (0.12599593089508843) [Z1 Z2]
+ (0.134426775029246) [Z0 Z1]
+ (0.1597246656203749) [Z2 Z3]
+ (-0.003489178165608661) [Y0 Z1 Y2]
+ (-0.003489178165608661) [X0 Z1 X2]
+ (-0.003489178165608661) [Y1 Z2 Y3]
+ (-0.003489178165608661) [X1 Z2 X3]
+ (-0.007764811135414218) [Y0 Y1 X2 X3]
+ (-0.007764811135414218) [X0 X1 Y2 Y3]
+ (0.0003125962232503304) [Z0 Y1 Z2 Y3]
+ (0.0003125962232503304) [Z0 X1 Z2 X3]
+ (0.00380177503471649) [Y0 Z1 Y2 Z3]
+ (0.00380177503471649) [X0 Z1 X2 Z3]
+ (0.007764811135414218) [Y0 X1 X2 Y3]
+ (0.007764811135414218) [X0 Y1 Y2 X3]


In [6]:
print(qubits)

4


In [7]:
H.num_params

27

In [8]:
H.ops

[Identity(wires=[0]),
 PauliZ(wires=[0]),
 PauliY(wires=[0]) @ PauliZ(wires=[1]) @ PauliY(wires=[2]),
 PauliX(wires=[0]) @ PauliZ(wires=[1]) @ PauliX(wires=[2]),
 PauliZ(wires=[2]),
 PauliZ(wires=[0]) @ PauliZ(wires=[2]),
 PauliZ(wires=[1]),
 PauliZ(wires=[0]) @ PauliZ(wires=[1]),
 PauliY(wires=[0]) @ PauliY(wires=[2]),
 PauliX(wires=[0]) @ PauliX(wires=[2]),
 PauliY(wires=[1]) @ PauliZ(wires=[2]) @ PauliY(wires=[3]),
 PauliZ(wires=[0]) @ PauliY(wires=[1]) @ PauliZ(wires=[2]) @ PauliY(wires=[3]),
 PauliX(wires=[1]) @ PauliZ(wires=[2]) @ PauliX(wires=[3]),
 PauliZ(wires=[0]) @ PauliX(wires=[1]) @ PauliZ(wires=[2]) @ PauliX(wires=[3]),
 PauliY(wires=[0]) @ PauliX(wires=[1]) @ PauliX(wires=[2]) @ PauliY(wires=[3]),
 PauliY(wires=[0]) @ PauliY(wires=[1]) @ PauliX(wires=[2]) @ PauliX(wires=[3]),
 PauliX(wires=[0]) @ PauliX(wires=[1]) @ PauliY(wires=[2]) @ PauliY(wires=[3]),
 PauliX(wires=[0]) @ PauliY(wires=[1]) @ PauliY(wires=[2]) @ PauliX(wires=[3]),
 PauliZ(wires=[3]),
 PauliZ(wires=[0])

In [9]:
H.sparse_matrix

<bound method Hamiltonian.sparse_matrix of <Hamiltonian: terms=27, wires=[0, 1, 2, 3]>>

In [10]:
H.terms()

([tensor(-109.09386199, requires_grad=False),
  tensor(0.03624012, requires_grad=False),
  tensor(-0.00348918, requires_grad=False),
  tensor(-0.00348918, requires_grad=False),
  tensor(-0.18501661, requires_grad=False),
  tensor(0.11823112, requires_grad=False),
  tensor(0.03624012, requires_grad=False),
  tensor(0.13442678, requires_grad=False),
  tensor(0.0003126, requires_grad=False),
  tensor(0.0003126, requires_grad=False),
  tensor(-0.00348918, requires_grad=False),
  tensor(0.0003126, requires_grad=False),
  tensor(-0.00348918, requires_grad=False),
  tensor(0.0003126, requires_grad=False),
  tensor(0.00776481, requires_grad=False),
  tensor(-0.00776481, requires_grad=False),
  tensor(-0.00776481, requires_grad=False),
  tensor(0.00776481, requires_grad=False),
  tensor(-0.18501661, requires_grad=False),
  tensor(0.12599593, requires_grad=False),
  tensor(0.00380178, requires_grad=False),
  tensor(0.00380178, requires_grad=False),
  tensor(0.11823112, requires_grad=False),
  te

In [11]:
print(qml.draw(H, decimals=None))

<function draw.<locals>.wrapper at 0x7f12145f0ee0>


H

##### Generate Excitations

In [12]:
singles, doubles = qchem.excitations(active_electrons, qubits)

In [13]:
singles

[[0, 2], [1, 3]]

In [14]:
doubles

[[0, 1, 2, 3]]

In [15]:
print(f"Total number of excitations = {len(singles) + len(doubles)}")

Total number of excitations = 3


In [16]:
singles_excitations = [qml.SingleExcitation(0.0, x) for x in singles]
singles_excitations

[SingleExcitation(0.0, wires=[0, 2]), SingleExcitation(0.0, wires=[1, 3])]

In [17]:
doubles_excitations = [qml.DoubleExcitation(0.0, x) for x in doubles]
doubles_excitations

[DoubleExcitation(0.0, wires=[0, 1, 2, 3])]

In [18]:
operator_pool = doubles_excitations + singles_excitations
operator_pool

[DoubleExcitation(0.0, wires=[0, 1, 2, 3]),
 SingleExcitation(0.0, wires=[0, 2]),
 SingleExcitation(0.0, wires=[1, 3])]

In [19]:
hf_state = qchem.hf_state(active_electrons, qubits)
hf_state

tensor([1, 1, 0, 0], requires_grad=True)

### Adaptive Optimizer

##### Helper Functions

In [23]:
from typing import Sequence, Callable
from pennylane import transforms

@transforms
def append_gate(tape: QuantumTape, params, gates) -> (Sequence[QuantumTape], Callable):
    """Append parameterized gates to an existing tape.

    Args:

        tape (QuantumTape or QNode or Callable): quantum circuit to transform by adding gates
        params (array[float]): parameters of the gates to be added
        gates (list[Operator]): list of the gates to be added

    Returns:
        qnode (QNode) or quantum function (Callable) or tuple[List[QuantumTape], function]: The transformed circuit as described in :func:`qml.transform <pennylane.transform>`.

    """
    new_operations = []

    for i, g in enumerate(gates):
        g = copy.copy(g)
        new_params = (params[i], *g.data[1:])
        g.data = new_params
        new_operations.append(g)

    new_tape = type(tape)(tape.operations + new_operations, tape.measurements, shots=tape.shots)

    def null_postprocessing(results):
        """A postprocesing function returned by a transform that only converts the batch of results
        into a result for a single ``QuantumTape``.
        """
        return results[0]  # pragma: no cover

    return [new_tape], null_postprocessing



TypeError: 'module' object is not callable

##### Adaptive Optimizer Class

In [ ]:
class AdaptiveOptimizer:
    def __init__(self, param_steps=10, stepsize=0.5):
        self.param_steps = param_steps
        self.stepsize = stepsize
    
    @staticmethod
    def _circuit(params, gates, initial_circuit):
        final_circuit = append_gate(initial_circuit, params, gates)
        return final_circuit()
    
    def step(self, circuit, operator_pool, params_zero=True):
        return self.step_and_cost(circuit, operator_pool, params_zero=params_zero)
    
    def step_and_cost(self, circuit, operator_pool, drain_pool = False, params_zero=True):
        cost = circuit()
        qnode = copy.copy(circuit)

        if drain_pool:
            operator_pool = [
                gate
                for gate in operator_pool
                if all(
                    gate.name != operation.name or gate.wires != operation.wires
                    for operation in circuit.tape.operations
                )
            ]
        
        params = np.array([gate.parameters[0] for gate in operator_pool])
        qnode.func = self._circuit
        grads = qml.grad(qnode)(params, gates=operator_pool, initial_circuit=circuit.func)

        selected_gates = [operator_pool[np.argmax(abs(grads))]]
        optimizer = qml.GradientDescentOptimizer(stepsize=self.stepsize)

        if params_zero:
            params = np.zeros(len(selected_gates))
        else:
            params = np.array([gate.parameters[0] for gate in selected_gates], requires_grad = True)
        
        for _ in range(self.param_steps):
            params, _ = optimizer.step_and_cost(
                qnode, params, gates=selected_gates, initial_circuit=circuit.func
            )
        
        qnode.func = append_gate(circuit.func, params, selected_gates)

        return qnode, cost, max(abs(qml.math.toarray(grads)))